In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow import keras
import tensorflow as tf

physical_devices = tf.config.list_physical_devices('GPU')
for device in physical_devices:
  tf.config.experimental.set_memory_growth(device, True)

In [2]:
# gpu 연결 체크
tf.test.is_gpu_available(
    cuda_only=False,
    min_cuda_compute_capability=None
)
# True

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [3]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range = 360,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip=True,
                                   vertical_flip = True,
                                   fill_mode = 'nearest')


train_generator = train_datagen.flow_from_directory(
                  'resample',
                  target_size = (150, 150),
                  batch_size = 16,
                  class_mode = 'sparse')

for data_batch, labels_batch in train_generator:
    # 데이터 하나만 체크해보자
    print('배치 데이터 크기:', data_batch.shape)
    print('배치 레이블 크기:', labels_batch.shape)
    break


Found 20108 images belonging to 4 classes.
배치 데이터 크기: (16, 150, 150, 3)
배치 레이블 크기: (16,)


In [4]:
inputs = keras.Input(shape=(150, 150, 3), name="img")
x = layers.Conv2D(16, 3, 2, padding = 'same', activation="relu")(inputs)
x = layers.Conv2D(32, 3, 1, padding ='same', activation="relu")(x)
x = layers.MaxPooling2D(3)(x)
x = layers.Conv2D(64, 3, 1, activation="relu", padding="same")(x)
x = layers.Conv2D(64, 3, 2, activation="relu", padding="same")(x)
x = layers.Flatten()(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(512, activation="relu")(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dense(32, activation="relu")(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(4)(x)
outputs = layers.Softmax()(x)

model = keras.Model(inputs, outputs, name="converea")
model.summary()

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=0.1),
    metrics=["accuracy"]
)


Model: "converea"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 img (InputLayer)            [(None, 150, 150, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 75, 75, 16)        448       
                                                                 
 conv2d_1 (Conv2D)           (None, 75, 75, 32)        4640      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 25, 25, 32)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 25, 25, 64)        18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 13, 13, 64)        36928     
                                                          

In [5]:
model.fit(train_generator, batch_size=16, epochs=2)
tf.saved_model.save(model, './model/converea')

Epoch 1/2
1257/1257 [==============================] - 111s 84ms/step - loss: 9.4120 - accuracy: 0.3560
Epoch 2/2
1257/1257 [==============================] - 104s 82ms/step - loss: 1.3346 - accuracy: 0.3584


INFO:tensorflow:Assets written to: ./model/converea\assets


INFO:tensorflow:Assets written to: ./model/converea\assets


In [6]:
model.save_weights('./model/checkpoint')

In [7]:
loaded = tf.saved_model.load('./model/converea')

loaded(data_batch)

<tf.Tensor: shape=(16, 4), dtype=float32, numpy=
array([[0.15329042, 0.34141153, 0.32754564, 0.17775244],
       [0.15329042, 0.34141153, 0.32754564, 0.17775244],
       [0.15329042, 0.34141153, 0.32754564, 0.17775244],
       [0.15329042, 0.34141153, 0.32754564, 0.17775244],
       [0.15329042, 0.34141153, 0.32754564, 0.17775244],
       [0.15329042, 0.34141153, 0.32754564, 0.17775244],
       [0.15329042, 0.34141153, 0.32754564, 0.17775244],
       [0.15329042, 0.34141153, 0.32754564, 0.17775244],
       [0.15329042, 0.34141153, 0.32754564, 0.17775244],
       [0.15329042, 0.34141153, 0.32754564, 0.17775244],
       [0.15329042, 0.34141153, 0.32754564, 0.17775244],
       [0.15329042, 0.34141153, 0.32754564, 0.17775244],
       [0.15329042, 0.34141153, 0.32754564, 0.17775244],
       [0.15329042, 0.34141153, 0.32754564, 0.17775244],
       [0.15329042, 0.34141153, 0.32754564, 0.17775244],
       [0.15329042, 0.34141153, 0.32754564, 0.17775244]], dtype=float32)>

In [ ]:
model.fit_generator(train_generator, batch_size=16, epochs=2, use_multiprocessing=True,
                         workers=6)
tf.saved_model.save(model, './model/converea')

In [3]:
import os
import json
os.environ['TF_CONFIG'] = json.dumps({
    'cluster': {
        'worker': ["localhost:12345", "localhost:23456"]
    },
    'task': {'type': 'worker', 'index': 0}
})

In [5]:
from tensorflow import keras
import tensorflow as tf

In [ ]:
strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

Instructions for updating:
use distribute.MultiWorkerMirroredStrategy instead


In [ ]:
NUM_WORKERS = 4
# `tf.data.Dataset.batch`에는 전역 배치 크기를 지정해야 하기 때문에
# 여기서 배치 크기는 워커의 수를 곱한 크기로 늘려야 합니다. 
# 전에는 64였지만, 이제 128이 됩니다.
GLOBAL_BATCH_SIZE = 64 * NUM_WORKERS
train_datasets = train_generator.batch(GLOBAL_BATCH_SIZE)
with strategy.scope():
    multi_worker_model = build_and_compile_cnn_model()
multi_worker_model.fit(x=train_datasets, epochs=2)

# model.fit(train_generator, batch_size=16, epochs=2)
tf.saved_model.save(multi_worker_model, './model/converea1')
